In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
import csv

In [2]:

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.107 Safari/537.36'}

In [3]:
base_url = 'https://www.themoviedb.org/movie?page='

all_page = []

for item in range(51):
    all_page.append(base_url+str(item))

In [4]:
#all_page

In [5]:
all_movie_data = []

for data in all_page:
    source = requests.get(data,headers=headers).text
    common = 'https://www.themoviedb.org'
    soup = bs(source,'lxml')
    div = soup.find_all('div',class_='image')
    for item in div:
        link = item.find('a')['href']
        combine = (common+str(link))
        source_1 = requests.get(combine,headers=headers).text
        soup_1 = bs(source_1,'lxml')
        div_1 = soup_1.find_all('div',class_='single_column')
        div_2 = soup_1.find_all('li',class_='profile')    
        cast = (combine+'/cast')
        cast_source_data1 = requests.get(cast,headers=headers).text
        cast_soup = bs(cast_source_data1,'lxml')
        cast_div = cast_soup.find_all('div',class_='crew_wrapper')
        for item_1 in div_1:
            try:
                movie = item_1.find('h2').text.split('(')[0].strip()
            except:
                movie = ('Null')

            try:
                rating = item_1.find('div',class_='user_score_chart')['data-percent']
            except:
                rating = ('Null')

            try:
                genre = item_1.find('span',class_='genres').text.strip()
            except:
                genre = ('Null')
            try:
                release = item_1.find('span',class_='release').text.strip().split('(')[0]
            except:
                release = ('Null')

            try:
                runtime = item_1.find('span',class_='runtime').text.strip()
            except:
                runtime = ('Null')

            try:
                url_1 = item_1.find('h2')
                url = url_1.find('a')['href']
                movie_url = (common+str(url))
            except:
                movie_url = ('Null')

            try:
                for i in cast_div:
                    dr = i.find('h4').text
                    if 'Directing' == dr:
                        d = i.find('div',class_='info')
                        director = d.find('a').text
            except:
                director = 'Null'

        data = {
            'Movie' : movie,
            'Rating' : rating,
            'Genre' : genre,
            'Release Date' : release,
            'Runtime' : runtime,
            'Director' : director,
            'URL' : movie_url
            }
        all_movie_data.append(data)

In [6]:
df = pd.DataFrame(all_movie_data)

In [7]:
df.to_csv('movielist.csv')